In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
"""!pip install yfinance
!pip install finta
!pip install chart_studio
!pip install --upgrade ta
!pip install pyngrok
!pip install flask-ngrok
!pip install flask"""

!ngrok authtoken "28adn3vL54F2hMApqXFZd9AztRA_VCTWGyLZjXXdhN4u7h8J"

In [3]:
import yfinance as yf
import seaborn as sns
#from yahoo_fin import stock_info
import datetime
import pandas as pd
import numpy as np
from finta import TA
from ta import add_all_ta_features
from ta.utils import dropna
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema


import chart_studio.plotly as py
from plotly.graph_objs import * 
import plotly.tools as tls
import chart_studio.tools as ctls
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

#from yahoo_fin import stock_info
from plotly.offline import plot, init_notebook_mode
init_notebook_mode()
import cufflinks as cf
cf.set_config_file(offline=True)

import json
import plotly
import plotly.express as px
import plotly.graph_objects as go

import logging
logging.getLogger().setLevel(logging.ERROR)

#importing the required libraries for deployment
from flask import Flask, render_template, request, redirect, url_for
from joblib import load
from flask_ngrok import run_with_ngrok
#from get_tweets import get_related_tweets
from logging import FileHandler,WARNING

%matplotlib inline

# Get Stock Data

In [5]:
# get stock data
def get_stock_data(stock):
    global df, symbol
    NUM_DAYS = 729  # The number of days of historical data to retrieve
    INTERVAL = '1h'     # Sample rate of historical data
    symbol = str(stock)  #str(input("Please enter stock symbol: "))     # Symbol of the desired stock

    #define start & dates
    start = (datetime.date.today() - datetime.timedelta( NUM_DAYS ) )
    end = datetime.datetime.today()
    
    #pull data
    df = yf.download(symbol, start=start, end=end, interval = INTERVAL)
    df.drop(["Adj Close", "Volume"], axis = 1, inplace = True)
    df.index.name = "time"
    #df.reset_index(inplace = True)
    #df.rename(columns = {"index" : "time"}, inplace = True)
    #df = df.set_index(pd.to_datetime(df["time"]))
    #df.drop(["time"], axis = 1, inplace = True)
    return df

In [6]:
df = get_stock_data("AUDUSD=X")
df.index = df.index.tz_localize(None)
df

# Get High_Low Positions

In [7]:
#get High and low positions
def get_max_min(prices, smoothing, window_range):
    smooth_prices = prices['Close'].rolling(window=smoothing).mean().dropna()
    local_max = argrelextrema(smooth_prices.values, np.greater)[0]
    local_min = argrelextrema(smooth_prices.values, np.less)[0]
    price_local_max_dt = []
    for i in local_max:
        if (i>window_range) and (i<len(prices)-window_range):
            price_local_max_dt.append(prices.iloc[i-window_range:i+window_range]['Close'].idxmax())
    price_local_min_dt = []
    for i in local_min:
        if (i>window_range) and (i<len(prices)-window_range):
            price_local_min_dt.append(prices.iloc[i-window_range:i+window_range]['Close'].idxmin())  
    maxima = pd.DataFrame(prices.loc[price_local_max_dt])
    minima = pd.DataFrame(prices.loc[price_local_min_dt])
    max_min = pd.concat([maxima, minima]).sort_index()
    max_min.index.name = "date"
    max_min = max_min.reset_index()
    max_min = max_min[~max_min.date.duplicated()]
    p = prices.reset_index() 
    max_min['day_num'] = p[p['time'].isin(max_min.date)].index.values
    max_min = max_min.set_index('day_num')['Close']
    
    return max_min


In [8]:
smoothing = 21
window = 500

minmax = get_max_min(df, smoothing, window)
minmax

# Create new dataframe

In [9]:
#Create new dataframe
res = df.copy()
res = res.reset_index()
res["minmax"] = minmax
res = res.set_index("time")
res

In [10]:
res.dropna()

# Plot 1

In [11]:
plt.figure(figsize = [20, 7])
res["Close"].plot()
#df.reset_index()["Close"].plot()
#plt.scatter(minmax.index, minmax.values, color = "red", alpha = .5)
res["minmax"].plot(linestyle='dotted', linewidth=1, marker='o')

# Plot 2

In [12]:
"""
fig = go.Figure(data=[go.Candlestick(x=df.index,
                    open=df['Open'],
                    high=df['High'],
                    low=df['Low'],
                    close=df['Close'])])
fig.add_trace(go.Scatter(x=res.index, y= result["minmax"].interpolate(),
                      line={'color':'blue', 'width':1},     
                      mode='lines',
                      name='high-low'))
fig.add_trace(
        go.Scatter(mode = "markers",
            x=result.index,
            y=result.minmax
        ))

fig.update_xaxes(
        rangebreaks=[
            dict(bounds=["sat", "mon"]), #hide weekends
            #dict(values=["2015-12-25", "2016-01-01"])  # hide Christmas and New Year's
        ]
    )

fig.update_layout(title=f'HIGH_LOW plot of {symbol}',
                   xaxis_title='index',
                   yaxis_title='Price',
                  xaxis_rangeslider_visible=False)

fig.show()
"""

# Automate - Identifying Highs & Lows

In [13]:
def get_highs_lows(instrument, window):
    #get stock data
    def get_stock_data():
        global df, symbol
        NUM_DAYS = 30 # The number of days of historical data to retrieve
        INTERVAL = '1h'     # Sample rate of historical data
        symbol = str(instrument) #str(input("Please enter stock symbol: "))   #str(stock)  # Symbol of the desired stock

        #define start & dates
        start = (datetime.date.today() - datetime.timedelta( NUM_DAYS ) )
        end = datetime.datetime.today()

        #pull data
        df = yf.download(symbol, start=start, end=end, interval = INTERVAL)
        df.drop(["Adj Close", "Volume"], axis = 1, inplace = True)
        df.index = df.index.tz_localize(None)
        df.index.name = "time"
        #df.reset_index(inplace = True)
        #df.rename(columns = {"index" : "time"}, inplace = True)
        #df = df.set_index(pd.to_datetime(df["time"]))
        #df.drop(["time"], axis = 1, inplace = True)
        return df
    
    get_stock_data()


    #get High and low positions
    def get_max_min(prices, window):
        global max_min, maxmin
        smoothing = 21 #int(input("Please enter SMA: ")) #Gold - (5, 100)
        window_range = window #int(input("Please enter Lookback window: "))

        smooth_prices = prices['Close'].rolling(window=smoothing).mean().dropna()
        local_max = argrelextrema(smooth_prices.values, np.greater)[0]
        local_min = argrelextrema(smooth_prices.values, np.less)[0]
        price_local_max_dt = []
        for i in local_max:
            if (i>window_range) and (i<len(prices)-window_range):
                price_local_max_dt.append(prices.iloc[i-window_range:i+window_range]['Close'].idxmax())
        price_local_min_dt = []
        for i in local_min:
            if (i>window_range) and (i<len(prices)-window_range):
                price_local_min_dt.append(prices.iloc[i-window_range:i+window_range]['Close'].idxmin())  
        maxima = pd.DataFrame(prices.loc[price_local_max_dt])
        minima = pd.DataFrame(prices.loc[price_local_min_dt])
        max_min = pd.concat([maxima, minima]).sort_index()
        max_min.index.name = "date"
        max_min = max_min.reset_index()
        max_min = max_min[~max_min.date.duplicated()]
        p = prices.reset_index() 
        max_min['day_num'] = p[p['time'].isin(max_min.date)].index.values
        max_min = max_min.set_index('day_num')['Close']
        
        #print(max_min)
        return max_min
    maxmin = get_max_min(df, window)
    
    #Create new dataframe
    res = df.copy()
    res = res.reset_index()
    res["maxmin"] = maxmin
    res = res.set_index("time", drop = True)
    res
    
    """"
    #Plot result
    plt.figure(figsize = [20, 7])
    plt.title(f"HIGH-LOW plot for {symbol}")
    res["Close"].plot()
    #df.reset_index()["Close"].plot()
    #plt.scatter(minmax.index, minmax.values, color = "red", alpha = .5)
    res["maxmin"].plot(linestyle='dashed', linewidth=1, marker='o')
    
    """
    
    def plot(res):
        global fig, graphJSON
        fig = go.Figure(data=[go.Candlestick(x=res.index,
                    open=res['Open'],
                    high=res['High'],
                    low=res['Low'],
                    close=res['Close'])])
        fig.add_trace(go.Scatter(x=res.index, y= res["maxmin"].interpolate(),
                      line={'color':'blue', 'width':1},     
                      mode='lines',
                      name='high-low'))
        fig.add_trace(
            go.Scatter(mode = "markers",
            x=res.index,
            y=res.maxmin
            ))

        fig.update_xaxes(
            rangebreaks=[
            dict(bounds=["sat", "mon"]), #hide weekends
            #dict(values=["2015-12-25", "2016-01-01"])  # hide Christmas and New Year's
            ]
        )

        fig.update_layout(title=f'HIGH_LOW plot of {symbol}',
                   xaxis_title='Date',
                   yaxis_title='Price',
                   xaxis_rangeslider_visible=False)

        fig.show()
        
        graphJSON = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
    
    plot(res)

    #py.plot(fig, filename = 'XAU ATR Hour Prediction Plot', auto_open = False)

get_highs_lows("EURUSD=X", 100)


In [15]:
get_highs_lows("AUDCHF=X", 21)

## FLASK DEPLOYMENT

In [16]:
import os
import threading

from flask import Flask
from pyngrok import ngrok

# Open a HTTP tunnel on the default port 80
public_url = ngrok.connect(port = '4040')

os.environ["FLASK_ENV"] = "development"

app = Flask(__name__, template_folder = "../input/templates")
port = 5000

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# ... Update inbound traffic via APIs to use the public-facing ngrok URL



# Define Flask routes
# render default webpage
@app.route('/')
def home():
    return render_template('multiple button.html')

# when the post method detect, then redirect to success function
@app.route('/', methods=['POST', 'GET'])
def get_data():
    global inst, wind
    if request.method == 'POST':
        inst = request.form['Iname']
        wind = int(request.form['window'])
        return redirect(url_for('success', name=inst))
    else:
      return render_template('multiple button.html')

# get the data for the requested query
@app.route('/success/<name>')
def success(name):
    stock_name = str(inst)
    get_highs_lows(stock_name, wind)
    
    #return redirect(url_for('success', name=stop, variable = lot))
    return render_template("highs lows.html", graphJSON=graphJSON)

In [17]:
# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()

In [18]:
stop

In [ ]:
ngrok.kill()